#### Day07: Lab

This lab focuses on extracting and displaying information from a database. 

Please run all the code chunks until you see the lab exercises (at the very bottom). 

In [1]:
!pip3 install sqlalchemy
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, and_, or_
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import func

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Setting things up:
engine = sqlalchemy.create_engine('sqlite:///geog.db')
Base = declarative_base() 

In [3]:
# Schemas
class Region(Base):
  __tablename__ = 'regions'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  departments = relationship("Department", backref = "region")

  def __init__(self, name):
    self.name = name 

  def __repr__(self):
    return "<Region('%s')>" % self.id 

In [4]:
class Department(Base):
  __tablename__ = 'departments'

  id = Column(Integer, primary_key=True)
  deptname = Column(String)
  region_id = Column(Integer, ForeignKey('regions.id')) 
  towns = relationship("Town", backref = "department")

  def __init__(self, deptname):
    self.deptname = deptname 

  def __repr__(self):
    return "<Department('%s')>" % self.id 

In [5]:
class Town(Base):
  __tablename__ = 'towns'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  population = Column(Integer)
  dept_id = Column(Integer, ForeignKey('departments.id'))

  def __init__(self, name, population):
    self.name = name 
    self.population = population

  def __repr__(self):
    return "<Town('%s')>" % (self.name)

In [6]:
#First time create tables
Base.metadata.create_all(engine) 

In [7]:
#Create a session to actually store things in the db
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
# Create regions
reg1 = Region('Region 1')
reg2 = Region('Region 2')
reg3 = Region('Region 3')

In [9]:
# Create departments, nested in regions
dept1 = Department('Department 1')
reg1.departments.append(dept1)

dept2 = Department('Department 2')
reg1.departments.append(dept2)

dept3 = Department('Department 3')
reg3.departments.append(dept3)

dept4 = Department('Department 4')
reg2.departments.append(dept4)


In [10]:
# Create towns, nested in departments
t1 = Town("a", 10000)
t2 = Town("b", 20000)
dept1.towns = [t1, t2]

t3 = Town("c", 30000)
t4 = Town("d", 40000)
dept2.towns = [t3, t4]

t5 = Town("e", 50000)
t6 = Town("f", 60000)
dept3.towns = [t5, t6]

t7 = Town("g", 70000)
t8 = Town("h", 80000)
dept4.towns = [t7, t8]

In [11]:
session.add_all([reg1, reg2, reg3])
session.add_all([dept1, dept2, dept3, dept4])
session.add_all([t1, t2, t3, t4, t5, t6, t7, t8])

session.commit()

1. Print all of the towns, and order them by Town.id in descending order.

In [25]:
for town in session.query(Town).order_by(Town.id.desc()):
    print(town.name, town.id)

AttributeError: 'InstanceState' object has no attribute '_post_inspect'

2. Display, by department, the cities having more than 50,000 inhabitants.

In [21]:
for Town in session.query(Town).joing(Department).filter(Town.population > 50000).order_by(Department.id.desc()):
    print(town.id, town.name, town.population, town.department.deptname)

AttributeError: 'InstanceState' object has no attribute '_post_inspect'

3. Display the total number of inhabitants per department using only a query. (no lists!)

In [29]:

for town in session.query(Department.deptname, func.sum(Town.population).label("sum_inhab")).join(Department).order_by(Department):
    print(town.deptname, town.sum_inhab)

ArgumentError: ORDER BY expression expected, got <class '__main__.Department'>.